In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import re
import pickle
from tqdm import tqdm
import os
from collections import Counter

from tensorflow.keras.layers import Dense,Input,Activation,Dropout,Flatten,Embedding, LSTM, Concatenate, Conv1D, BatchNormalization
from tensorflow.keras.models import Model
import random as rn
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [52]:
from flask import Flask, jsonify, request
from bs4 import BeautifulSoup 

In [2]:

from sklearn.metrics import recall_score, f1_score, roc_auc_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [53]:
import flask
app=Flask(__name__)


In [4]:
df_pat_nb = pd.read_csv('drive/MyDrive/encounters.csv')
df_pat_nb.head()

,Unnamed: 0,patient_nbr,count_encounter
0,0,88785891,40
1,1,43140906,28
2,2,1660293,23
3,3,88227540,23
4,4,23199021,23


In [45]:
@app.route("/predict", methods=['POST'])
def predict(x):
    
    #age extraction:
    x['age'] = x['age'].replace('[','')
    x['age'] = x['age'].replace(')','')
    l = x['age'].split('-')
    x['age'] = (int(l[0]) + int(l[1]))/2

    #imputation of race:
    if x['race'] =='?':
        x['race'] == 'Hispanic'
    
    #dropping some features:
    feat_to_drop = ['weight', 'payer_code','medical_specialty', 'diag_1', 'diag_2','diag_3']
    x_new = {k:v for k,v in x.items() if k not in feat_to_drop}
    
    #creating new features:
    x_new['procedure_per_med'] = (x_new['num_lab_procedures'] + x_new[ 'num_procedures'])/x_new['num_medications']
    
    #creating some more new features:
    x_new['procedure_per_med'] = (x_new['num_lab_procedures'] + x_new[ 'num_procedures'])/x_new['num_medications']
    
    meds = ['metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton',
       'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone']
    
    #num_med_up = count of medicine whose dosage were upgraded
    #num_med_used = total med - meds which were not prescribed = (23 - meds with 'NO' tag)
    count_up = 0
    count_no = 0
    for med in meds:
        if x_new[med] == 'Up':
            count_up += 1
            count_no += 1
    x_new['num_med_up'] = count_up
    x_new['num_med_used'] = 23 - count_no
    
    #count_encounter = Number of encounters of each patient
    x_new['count_encounter'] = df_pat_nb[df_pat_nb['patient_nbr'] == x['patient_nbr']]['patient_nbr'].values[0]
    
    
    #creating gender dummies:
    if x_new['gender'] == 'Male':
        x_new_gender = [0,1,0]
            
    if x_new['gender'] == 'Female':
        x_new_gender = [1,0,0]
    else:
        x_new_gender = [0,0,1]
    
    #race_AfricanAmerican	race_Asian	race_Caucasian	race_Hispanic	race_Other
    
    if x_new['race'] == 'AfricanAmerican':
        x_new_race = [1,0,0,0,0]
        
    if x_new['race'] == 'Asian':
        x_new_race = [0,1,0,0,0]
        
    if x_new['race'] == 'Caucasian':
        x_new_race = [0,0,1,0,0]
        
    if x_new['race'] == 'Hispanic':
        x_new_race = [0,0,0,1,0]
        
    else:
        x_new_race = [0,0,0,0,1]
        
    #max_glu_serum_>200	max_glu_serum_>300	max_glu_serum_None	max_glu_serum_Norm
    if x_new['max_glu_serum'] == '>200':
        x_new_max_glu = [1,0,0,0]
    if x_new['max_glu_serum'] == '>300':
        x_new_max_glu = [0,1,0,0]
    if x_new['max_glu_serum'] == 'Norm':
        x_new_max_glu = [0,0,0,1]
    else:
        x_new_max_glu = [0,0,1,0]
        
    #'A1Cresult_>7', 'A1Cresult_>8', 'A1Cresult_None','A1Cresult_Norm'
    if x_new['A1Cresult'] == '>7':
        x_new_a1c = [1,0,0,0]
    if x_new['A1Cresult'] == '>8':
        x_new_a1c = [0,1,0,0]
    if x_new['A1Cresult'] == '>Norm':
        x_new_a1c = [0,0,0,1]
    else:
        x_new_a1c = [0,0,1,0]
    
    #'metformin_Down', 'metformin_No', 'metformin_Steady','metformin_Up'
    if x_new['metformin'] == 'Down':
        x_new_metfor = [1,0,0,0]
    elif x_new['metformin'] == 'No':
        x_new_metfor = [0,1,0,0]
    elif x_new['metformin'] == 'Steady':
        x_new_metfor = [0,0,1,0]
    else:
        x_new_metfor = [0,0,0,1]
    
    #'repaglinide_Down', 'repaglinide_No','repaglinide_Steady', 'repaglinide_Up'
    if x_new['repaglinide'] == 'Down':
        x_new_repag = [1,0,0,0]
    elif x_new['repaglinide'] == 'No':
        x_new_repag = [0,1,0,0]
    elif x_new['repaglinide'] == 'Steady':
        x_new_repag = [0,0,1,0]
    else:
        x_new_repag = [0,0,0,1]
        
    #'nateglinide_Down','nateglinide_No', 'nateglinide_Steady', 'nateglinide_Up'
    if x_new['nateglinide'] == 'Down':
        x_new_nateg = [1,0,0,0]
    elif x_new['nateglinide'] == 'No':
        x_new_nateg = [0,1,0,0]
    elif x_new['nateglinide'] == 'Steady':
        x_new_nateg = [0,0,1,0]
    else:
        x_new_nateg = [0,0,0,1]
        
    #'chlorpropamide_Down', 'chlorpropamide_No', 'chlorpropamide_Steady','chlorpropamide_Up'
    if x_new['chlorpropamide'] == 'Down':
        x_new_chlorpropamide = [1,0,0,0]
    elif x_new['chlorpropamide'] == 'No':
        x_new_chlorpropamide = [0,1,0,0]
    elif x_new['chlorpropamide'] == 'Steady':
        x_new_chlorpropamide = [0,0,1,0]
    else:
        x_new_chlorpropamide = [0,0,0,1]
        
    #'glimepiride_Down', 'glimepiride_No','glimepiride_Steady', 'glimepiride_Up'
    if x_new['glimepiride'] == 'Down':
        x_new_glimepiride = [1,0,0,0]
    elif x_new['glimepiride'] == 'No':
        x_new_glimepiride = [0,1,0,0]
    elif x_new['glimepiride'] == 'Steady':
        x_new_glimepiride = [0,0,1,0]
    else:
        x_new_glimepiride = [0,0,0,1]
        
    #'acetohexamide_No','acetohexamide_Steady'
    if x_new['acetohexamide'] == 'No':
        x_new_acetohexamide = [1,0]
    else:
        x_new_acetohexamide = [0,1]
        
    #'glipizide_Down', 'glipizide_No','glipizide_Steady', 'glipizide_Up'
    if x_new['glipizide'] == 'Down':
        x_new_glipizide = [1,0,0,0]
    elif x_new['glipizide'] == 'No':
        x_new_glipizide = [0,1,0,0]
    elif x_new['glipizide'] == 'Steady':
        x_new_glipizide = [0,0,1,0]
    else:
        x_new_glipizide = [0,0,0,1]
        
    #'glyburide_Down', 'glyburide_No','glyburide_Steady', 'glyburide_Up'
    if x_new['glyburide'] == 'Down':
        x_new_glyburide = [1,0,0,0]
    elif x_new['glyburide'] == 'No':
        x_new_glyburide = [0,1,0,0]
    elif x_new['glyburide'] == 'Steady':
        x_new_glyburide = [0,0,1,0]
    else:
        x_new_glyburide = [0,0,0,1]
        
    #'tolbutamide_No','tolbutamide_Steady'
    if x_new['tolbutamide'] == 'No':
        x_new_tolbutamide = [1,0]
    else:
        x_new_tolbutamide = [0,1]
    
    #'pioglitazone_Down', 'pioglitazone_No','pioglitazone_Steady', 'pioglitazone_Up'
    if x_new['pioglitazone'] == 'Down':
        x_new_pioglitazone = [1,0,0,0]
    elif x_new['pioglitazone'] == 'No':
        x_new_pioglitazone = [0,1,0,0]
    elif x_new['pioglitazone'] == 'Steady':
        x_new_pioglitazone = [0,0,1,0]
    else:
        x_new_pioglitazone = [0,0,0,1]  
        
    #'rosiglitazone_Down','rosiglitazone_No', 'rosiglitazone_Steady', 'rosiglitazone_Up'
    if x_new['rosiglitazone'] == 'Down':
        x_new_rosiglitazone = [1,0,0,0]
    elif x_new['rosiglitazone'] == 'No':
        x_new_rosiglitazone = [0,1,0,0]
    elif x_new['rosiglitazone'] == 'Steady':
        x_new_rosiglitazone = [0,0,1,0]
    else:
        x_new_rosiglitazone = [0,0,0,1] 
        
    #'acarbose_Down', 'acarbose_No', 'acarbose_Steady', 'acarbose_Up'
    if x_new['acarbose'] == 'Down':
        x_new_acarbose = [1,0,0,0]
    elif x_new['acarbose'] == 'No':
        x_new_acarbose = [0,1,0,0]
    elif x_new['acarbose'] == 'Steady':
        x_new_acarbose = [0,0,1,0]
    else:
        x_new_acarbose = [0,0,0,1]  
        
    # 'miglitol_Down', 'miglitol_No', 'miglitol_Steady', 'miglitol_Up'
    if x_new['miglitol'] == 'Down':
        x_new_miglitol = [1,0,0,0]
    elif x_new['miglitol'] == 'No':
        x_new_miglitol = [0,1,0,0]
    elif x_new['miglitol'] == 'Steady':
        x_new_miglitol = [0,0,1,0]
    else:
        x_new_miglitol = [0,0,0,1] 
        
    #'troglitazone_No', 'troglitazone_Steady'
    if x_new['troglitazone'] == 'No':
        x_new_troglitazone = [1,0]
    else:
        x_new_troglitazone = [0,1]
        
    #'tolazamide_No','tolazamide_Steady','tolazamide_Up'
    if x_new['tolazamide'] == 'No':
        x_new_tolazamide = [1,0,0]
    elif x_new['tolazamide'] == 'Steady':
        x_new_tolazamide = [0,1,0]
    else:
        x_new_tolazamide = [0,0,1] 
        
    #'examide_No', 'citoglipton_No'
    x_new_examide = [1]
    x_new_citoglipton = [1]
    
    #'insulin_Down', 'insulin_No', 'insulin_Steady', 'insulin_Up'
    if x_new['insulin'] == 'Down':
        x_new_insulin = [1,0,0,0]
    elif x_new['insulin'] == 'No':
        x_new_insulin = [0,1,0,0]
    elif x_new['insulin'] == 'Steady':
        x_new_insulin = [0,0,1,0]
    else:
        x_new_insulin = [0,0,0,1] 
        
    #'glyburide-metformin_Down', 'glyburide-metformin_No','glyburide-metformin_Steady', 'glyburide-metformin_Up'
    if x_new['glyburide-metformin'] == 'Down':
        x_new_glyburide_metformin = [1,0,0,0]
    elif x_new['glyburide-metformin'] == 'No':
        x_new_glyburide_metformin = [0,1,0,0]
    elif x_new['glyburide-metformin'] == 'Steady':
        x_new_glyburide_metformin = [0,0,1,0]
    else:
        x_new_glyburide_metformin = [0,0,0,1]
        
    #'glipizide-metformin_No', 'glipizide-metformin_Steady'
    if x_new['glipizide-metformin'] == 'No':
        x_new_glipizide_metformin = [1,0]
    else:
        x_new_glipizide_metformin = [0,1]  
        
    #'glimepiride-pioglitazone_No', 'glimepiride-pioglitazone_Steady'
    if x_new['glimepiride-pioglitazone'] == 'No':
        x_new_glimepiride_pioglitazone = [1,0]
    else:
        x_new_glimepiride_pioglitazone = [0,1]
    
    #'metformin-rosiglitazone_No', 'metformin-rosiglitazone_Steady'
    if x_new['metformin-rosiglitazone'] == 'No':
        x_new_metformin_rosiglitazone = [1,0]
    else:
        x_new_metformin_rosiglitazone = [0,1]
        
    #'metformin-pioglitazone_No', 'metformin-pioglitazone_Steady'
    if x_new['metformin-pioglitazone'] == 'No':
        x_new_metformin_pioglitazone = [1,0]
    else:
        x_new_metformin_pioglitazone = [0,1] 
        
    #'change_Ch', 'change_No', 'diabetesMed_No', 'diabetesMed_Yes'
    if x_new['change'] == 'Ch':
        x_new_change = [1,0]
    else:
        x_new_change = [0,1]
        
    if x_new['diabetesMed'] == 'No':
        x_new_diaMed = [1,0]
    else:
        x_new_diaMed = [0,1]

    features_num = ['age', 'admission_type_id', 'discharge_disposition_id',
       'admission_source_id', 'time_in_hospital', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'number_diagnoses',
       'procedure_per_med', 'num_med_up', 'num_med_used', 'count_encounter']

    features_cat = [x_new_gender, x_new_race, x_new_max_glu, x_new_a1c, x_new_metfor, x_new_repag,
                    x_new_nateg, x_new_chlorpropamide, x_new_glimepiride, x_new_acetohexamide,
                    x_new_glipizide, x_new_glyburide, x_new_tolbutamide, x_new_pioglitazone,
                    x_new_rosiglitazone, x_new_acarbose, x_new_miglitol, x_new_troglitazone,
                    x_new_tolazamide, x_new_examide, x_new_citoglipton, x_new_insulin,
                    x_new_glyburide_metformin, x_new_glipizide_metformin, x_new_glimepiride_pioglitazone,
                    x_new_metformin_rosiglitazone, x_new_metformin_pioglitazone, x_new_change,
                    x_new_diaMed]

    '''features_cat = ['gender_Female', 'gender_Male', 'gender_Unknown/Invalid',
       'race_AfricanAmerican', 'race_Asian', 'race_Caucasian', 'race_Hispanic',
       'race_Other', 'max_glu_serum_>200', 'max_glu_serum_>300',
       'max_glu_serum_None', 'max_glu_serum_Norm', 'A1Cresult_>7',
       'A1Cresult_>8', 'A1Cresult_None', 'A1Cresult_Norm', 'metformin_Down',
       'metformin_No', 'metformin_Steady', 'metformin_Up', 'repaglinide_Down',
       'repaglinide_No', 'repaglinide_Steady', 'repaglinide_Up',
       'nateglinide_Down', 'nateglinide_No', 'nateglinide_Steady',
       'nateglinide_Up', 'chlorpropamide_Down', 'chlorpropamide_No',
       'chlorpropamide_Steady', 'chlorpropamide_Up', 'glimepiride_Down',
       'glimepiride_No', 'glimepiride_Steady', 'glimepiride_Up',
       'acetohexamide_No', 'acetohexamide_Steady', 'glipizide_Down',
       'glipizide_No', 'glipizide_Steady', 'glipizide_Up', 'glyburide_Down',
       'glyburide_No', 'glyburide_Steady', 'glyburide_Up', 'tolbutamide_No',
       'tolbutamide_Steady', 'pioglitazone_Down', 'pioglitazone_No',
       'pioglitazone_Steady', 'pioglitazone_Up', 'rosiglitazone_Down',
       'rosiglitazone_No', 'rosiglitazone_Steady', 'rosiglitazone_Up',
       'acarbose_Down', 'acarbose_No', 'acarbose_Steady', 'acarbose_Up',
       'miglitol_Down', 'miglitol_No', 'miglitol_Steady', 'miglitol_Up',
       'troglitazone_No', 'troglitazone_Steady', 'tolazamide_No',
       'tolazamide_Steady', 'tolazamide_Up', 'examide_No', 'citoglipton_No',
       'insulin_Down', 'insulin_No', 'insulin_Steady', 'insulin_Up',
       'glyburide-metformin_Down', 'glyburide-metformin_No',
       'glyburide-metformin_Steady', 'glyburide-metformin_Up',
       'glipizide-metformin_No', 'glipizide-metformin_Steady',
       'glimepiride-pioglitazone_No', 'glimepiride-pioglitazone_Steady',
       'metformin-rosiglitazone_No', 'metformin-rosiglitazone_Steady',
       'metformin-pioglitazone_No', 'metformin-pioglitazone_Steady',
       'change_Ch', 'change_No', 'diabetesMed_No', 'diabetesMed_Yes']'''
    
    input_x = []
    for f in features_num:
      input_x.append(x_new[f])
    for f in features_cat:

      input_x.extend(f)
    
    #print('Length of input:',np.array(input_x).reshape(1,-1).shape)

    # load the model from disk
    loaded_model = pickle.load(open('drive/MyDrive/log_model.sav', 'rb'))
    output = loaded_model.predict(np.array(input_x).reshape(1,-1))


    #if output[0][0] == 1.0:
      #print('High chances of readmittance in less than 30 days.')
      
    #else:
      #print('No chance of readmdittance before 30 days.')
      
    return output[0]

In [47]:
def score(x,y):
  prediction = []
  target = []
  for i in range(len(x)):
    prediction.append(predict(x[i]))

  return ('Score:',f1_score(y, prediction, average='micro'))


In [48]:
x = [{'encounter_id':35754 , 'patient_nbr':82637451, 'race':'Caucasian', 'gender':'Male', 'age':'[50-60)', 'weight':'?',
       'admission_type_id':2, 'discharge_disposition_id':1,
       'admission_source_id':2, 'time_in_hospital':3, 'payer_code':'?',
       'medical_specialty':'?', 'num_lab_procedures':31, 'num_procedures':6,
       'num_medications':16, 'number_outpatient':0, 'number_emergency':0,
       'number_inpatient':0, 'diag_1':414, 'diag_2':411, 'diag_3':250,
       'number_diagnoses':9, 'max_glu_serum':'None', 'A1Cresult':'None', 'metformin':'No',
       'repaglinide':'No', 'nateglinide':'No', 'chlorpropamide':'No', 'glimepiride':'No',
       'acetohexamide':'No', 'glipizide':'No', 'glyburide':'No', 'tolbutamide':'No',
       'pioglitazone':'No', 'rosiglitazone':'No', 'acarbose':'No', 'miglitol':'No',
       'troglitazone':'No', 'tolazamide':'No', 'examide':'No', 'citoglipton':'No', 'insulin':'Steady',
       'glyburide-metformin':'No', 'glipizide-metformin':'No',
       'glimepiride-pioglitazone':'No', 'metformin-rosiglitazone':'No',
       'metformin-pioglitazone':'No', 'change':'No', 'diabetesMed':'Yes'},
     {'encounter_id': 623880 , 'patient_nbr':17928900, 'race':'Caucasian', 'gender':'Female', 'age':'[60-70)', 'weight':'?',
       'admission_type_id':2, 'discharge_disposition_id':1,
       'admission_source_id':2, 'time_in_hospital':6, 'payer_code':'?',
       'medical_specialty':'?', 'num_lab_procedures':58, 'num_procedures':3,
       'num_medications':14, 'number_outpatient':0, 'number_emergency':0,
       'number_inpatient':0, 'diag_1':182, 'diag_2':289, 'diag_3':250,
       'number_diagnoses':9, 'max_glu_serum':'None', 'A1Cresult':'None', 'metformin':'No',
       'repaglinide':'No', 'nateglinide':'No', 'chlorpropamide':'No', 'glimepiride':'No',
       'acetohexamide':'No', 'glipizide':'No', 'glyburide':'No', 'tolbutamide':'No',
       'pioglitazone':'No', 'rosiglitazone':'No', 'acarbose':'No', 'miglitol':'No',
       'troglitazone':'No', 'tolazamide':'No', 'examide':'No', 'citoglipton':'No', 'insulin':'Down',
       'glyburide-metformin':'No', 'glipizide-metformin':'No',
       'glimepiride-pioglitazone':'No', 'metformin-rosiglitazone':'No',
       'metformin-pioglitazone':'No', 'change':'Ch', 'diabetesMed':'Yes'},
     {'encounter_id':4262964 , 'patient_nbr':8308170, 'race':'Caucasian', 'gender':'Male', 'age':'[40-50)', 'weight':'?',
       'admission_type_id':6, 'discharge_disposition_id':25,
       'admission_source_id':4, 'time_in_hospital':3, 'payer_code':'?',
       'medical_specialty':'Cardiology', 'num_lab_procedures':44, 'num_procedures':6,
       'num_medications':12, 'number_outpatient':0, 'number_emergency':0,
       'number_inpatient':0, 'diag_1':414, 'diag_2':411, 'diag_3':250,
       'number_diagnoses':9, 'max_glu_serum':'None', 'A1Cresult':'>8', 'metformin':'No',
       'repaglinide':'No', 'nateglinide':'No', 'chlorpropamide':'No', 'glimepiride':'Steady',
       'acetohexamide':'No', 'glipizide':'No', 'glyburide':'No', 'tolbutamide':'No',
       'pioglitazone':'No', 'rosiglitazone':'No', 'acarbose':'No', 'miglitol':'No',
       'troglitazone':'No', 'tolazamide':'No', 'examide':'No', 'citoglipton':'No', 'insulin':'Steady',
       'glyburide-metformin':'No', 'glipizide-metformin':'No',
       'glimepiride-pioglitazone':'No', 'metformin-rosiglitazone':'No',
       'metformin-pioglitazone':'No', 'change':'Ch', 'diabetesMed':'Yes'},
     {'encounter_id':26997708 , 'patient_nbr':388008, 'race':'AfricanAmerican', 'gender':'Female', 'age':'[50-60)', 'weight':'?',
       'admission_type_id':3, 'discharge_disposition_id':1,
       'admission_source_id':4, 'time_in_hospital':7, 'payer_code':'?',
       'medical_specialty':'?', 'num_lab_procedures':41, 'num_procedures':1,
       'num_medications':20, 'number_outpatient':0, 'number_emergency':0,
       'number_inpatient':1, 'diag_1':'v57', 'diag_2':11, 'diag_3':250,
       'number_diagnoses':9, 'max_glu_serum':'None', 'A1Cresult':'None', 'metformin':'No',
       'repaglinide':'No', 'nateglinide':'No', 'chlorpropamide':'No', 'glimepiride':'No',
       'acetohexamide':'No', 'glipizide':'No', 'glyburide':'No', 'tolbutamide':'No',
       'pioglitazone':'No', 'rosiglitazone':'No', 'acarbose':'No', 'miglitol':'No',
       'troglitazone':'No', 'tolazamide':'No', 'examide':'No', 'citoglipton':'No', 'insulin':'No',
       'glyburide-metformin':'No', 'glipizide-metformin':'No',
       'glimepiride-pioglitazone':'No', 'metformin-rosiglitazone':'No',
       'metformin-pioglitazone':'No', 'change':'No', 'diabetesMed':'No'},
     {'encounter_id':36900 , 'patient_nbr':77391171, 'race':'AfricanAmerican', 'gender':'Male', 'age':'[60-70)', 'weight':'?',
       'admission_type_id':2, 'discharge_disposition_id':1,
       'admission_source_id':4, 'time_in_hospital':7, 'payer_code':'?',
       'medical_specialty':'?', 'num_lab_procedures':62, 'num_procedures':0,
       'num_medications':11, 'number_outpatient':0, 'number_emergency':0,
       'number_inpatient':0, 'diag_1':157, 'diag_2':288, 'diag_3':250,
       'number_diagnoses':7, 'max_glu_serum':'None', 'A1Cresult':'None', 'metformin':'No',
       'repaglinide':'No', 'nateglinide':'No', 'chlorpropamide':'No', 'glimepiride':'No',
       'acetohexamide':'No', 'glipizide':'No', 'glyburide':'No', 'tolbutamide':'No',
       'pioglitazone':'No', 'rosiglitazone':'No', 'acarbose':'No', 'miglitol':'No',
       'troglitazone':'No', 'tolazamide':'No', 'examide':'No', 'citoglipton':'No', 'insulin':'Steady',
       'glyburide-metformin':'No', 'glipizide-metformin':'No',
       'glimepiride-pioglitazone':'No', 'metformin-rosiglitazone':'No',
       'metformin-pioglitazone':'No', 'change':'Ch', 'diabetesMed':'Yes'},
     {'encounter_id':40926 , 'patient_nbr':85504905, 'race':'Caucasian', 'gender':'Female', 'age':'[40-50)', 'weight':'?',
       'admission_type_id':1, 'discharge_disposition_id':3,
       'admission_source_id':7, 'time_in_hospital':7, 'payer_code':'?',
       'medical_specialty':'?', 'num_lab_procedures':60, 'num_procedures':0,
       'num_medications':15, 'number_outpatient':0, 'number_emergency':1,
       'number_inpatient':0, 'diag_1':414, 'diag_2':411, 'diag_3':250,
       'number_diagnoses':8, 'max_glu_serum':'None', 'A1Cresult':'None', 'metformin':'Steady',
       'repaglinide':'Up', 'nateglinide':'No', 'chlorpropamide':'No', 'glimepiride':'No',
       'acetohexamide':'No', 'glipizide':'No', 'glyburide':'No', 'tolbutamide':'No',
       'pioglitazone':'No', 'rosiglitazone':'No', 'acarbose':'No', 'miglitol':'No',
       'troglitazone':'No', 'tolazamide':'No', 'examide':'No', 'citoglipton':'No', 'insulin':'Down',
       'glyburide-metformin':'No', 'glipizide-metformin':'No',
       'glimepiride-pioglitazone':'No', 'metformin-rosiglitazone':'No',
       'metformin-pioglitazone':'No', 'change':'Ch', 'diabetesMed':'Yes'},
     {'encounter_id':77076 , 'patient_nbr':92519352, 'race':'AfricanAmerican', 'gender':'Male', 'age':'[50-60)', 'weight':'?',
       'admission_type_id':1, 'discharge_disposition_id':1,
       'admission_source_id':7, 'time_in_hospital':4, 'payer_code':'?',
       'medical_specialty':'?', 'num_lab_procedures':45, 'num_procedures':4,
       'num_medications':17, 'number_outpatient':0, 'number_emergency':0,
       'number_inpatient':0, 'diag_1':414, 'diag_2':411, 'diag_3':250,
       'number_diagnoses':8, 'max_glu_serum':'None', 'A1Cresult':'None', 'metformin':'No',
       'repaglinide':'No', 'nateglinide':'No', 'chlorpropamide':'No', 'glimepiride':'No',
       'acetohexamide':'No', 'glipizide':'Steady', 'glyburide':'No', 'tolbutamide':'No',
       'pioglitazone':'No', 'rosiglitazone':'No', 'acarbose':'No', 'miglitol':'No',
       'troglitazone':'No', 'tolazamide':'No', 'examide':'No', 'citoglipton':'No', 'insulin':'Steady',
       'glyburide-metformin':'No', 'glipizide-metformin':'No',
       'glimepiride-pioglitazone':'No', 'metformin-rosiglitazone':'No',
       'metformin-pioglitazone':'No', 'change':'Ch', 'diabetesMed':'Yes'},
     {'encounter_id':55842 , 'patient_nbr':84259809, 'race':'Caucasian', 'gender':'Male', 'age':'[60-70)', 'weight':'?',
       'admission_type_id':3, 'discharge_disposition_id':1,
       'admission_source_id':2, 'time_in_hospital':4, 'payer_code':'?',
       'medical_specialty':'?', 'num_lab_procedures':70, 'num_procedures':1,
       'num_medications':21, 'number_outpatient':0, 'number_emergency':0,
       'number_inpatient':0, 'diag_1':414, 'diag_2':411, 'diag_3':250,
       'number_diagnoses':7, 'max_glu_serum':'None', 'A1Cresult':'None', 'metformin':'Steady',
       'repaglinide':'No', 'nateglinide':'No', 'chlorpropamide':'No', 'glimepiride':'Steady',
       'acetohexamide':'No', 'glipizide':'No', 'glyburide':'No', 'tolbutamide':'No',
       'pioglitazone':'No', 'rosiglitazone':'No', 'acarbose':'No', 'miglitol':'No',
       'troglitazone':'No', 'tolazamide':'No', 'examide':'No', 'citoglipton':'No', 'insulin':'Steady',
       'glyburide-metformin':'No', 'glipizide-metformin':'No',
       'glimepiride-pioglitazone':'No', 'metformin-rosiglitazone':'No',
       'metformin-pioglitazone':'No', 'change':'Ch', 'diabetesMed':'Yes'}]
y = [0,0, 0,1,1, 1, 1, 1 ]

In [49]:

print(score(x,y))

('Score:', 0.625)
